In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# import numpy as np

# indices = np.load('/content/gdrive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/train_data/train_shuffled_indices.npy').tolist()
# latter = []

# for i in indices[:264813 + 20000]:
#     if i > 19023:
#         latter.append(i)
# former = list(range(0, 19023 + 1))

# print(len(latter) + len(former))

# indices_300k = sorted(former + latter)

300807


In [3]:
import pandas as pd
path ="/content/gdrive/My Drive/Incivility/Perspective/Perspective_BERT/Final/Data/"
filename = "train.csv"

# path = '/content/gdrive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/train_data/'
# filename = 'toxicity.csv'

# path2 = '/content/gdrive/My Drive/Incivility/Perspective/Perspective_BERT/Retrained/'
# filename2 = 'replaced_toxicity.csv'

threshold = 0.5

In [ ]:
# !unzip "/content/gdrive/My Drive/Incivility/Data/train.csv.zip" -d "/content/gdrive/My Drive/Incivility/Data/"

In [4]:
train_data  = pd.read_csv(path+filename, usecols=['comment_text', 'target']) 
# additional_train_data = pd.read_csv(path2+filename2, usecols=['comment_text', 'target'])

In [ ]:
# print(train_data.shape)
# train_data = train_data.drop(indices_300k)
# print(train_data.shape)

# print(additional_train_data.shape)

(50087, 2)


In [5]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 4.3MB/s 
     |████████████████████████████████| 133kB 23.6MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 7.5MB 39.8MB/s 
ERROR: botocore 1.20.61 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [6]:
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 324684.55B/s]


In [8]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
from tqdm import tqdm, tqdm_notebook
import numpy as np
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [9]:
import pandas as pd
seed = 6666
train_size = 200000                         #Train size 
valid_size= 10000                          #Validation Size
MAX_SEQUENCE_LENGTH = 256
train_df = train_data.sample(train_size+valid_size, random_state=seed)
print('loaded %d records' % len(train_df))

# print(train_df.head())
# print(additional_train_data.head())

# Insert additional masked-task records over this one
# train_df = pd.concat([additional_train_data, train_df], ignore_index=True)
# print('Now there are %d ' % len(train_df))

print(train_df.head)
# Make sure all comment_text values are strings
train_df['comment_text'] = train_df['comment_text'].astype(str) 

sequences = convert_lines(train_df["comment_text"],MAX_SEQUENCE_LENGTH,tokenizer)
train_df=train_df.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']

train_df = train_df.drop(['comment_text'],axis=1)
# convert target to 0,1
train_df['target']=(train_df['target']>threshold).astype(float)

loaded 210000 records
<bound method NDFrame.head of          target                                       comment_text
1531413     0.1  Layla4,\n\nThe chances of all the opposition c...
30637       0.0  PFD is guaranteed for three years. I guess Gov...
1196639     0.0                    Wrong target, it should be MSM.
1638833     0.2  The duty on imported cheese is something like ...
1456330     0.0  I think they call it priorities JPWPG. You can...
...         ...                                                ...
9207        0.0  The Donkey Party has a long history of politic...
659352      0.0  Kenai River and Cook Inlet use should be drama...
701484      0.0  It's not possible this even happened...that's ...
993704      0.0  What is Colt Lyerla's base salary? What will E...
1288834     0.0  Re the MADD tie-in and "They were simply again...

[210000 rows x 2 columns]>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



269


In [10]:
X = sequences[:train_size]                
y = train_df[y_columns].values[:train_size]
X_val = sequences[train_size:]                
y_val = train_df[y_columns].values[train_size:]

In [11]:
test_df=train_df.tail(valid_size).copy()
train_df=train_df.head(train_size) 

In [12]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))

In [13]:
path = "/content/gdrive/My Drive/Incivility/Perspective/Perspective_BERT/bert_pytorch_0.5_larger.bin"
EPOCHS = 1
lr=2e-5
batch_size = 32
accumulation_steps=1
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

device=torch.device('cuda')

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",cache_dir=None,num_labels=len(y_columns))
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset
num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)

model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    for i,(x_batch, y_batch) in tk0:
        optimizer.zero_grad()
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        loss.backward()
        if (i+1) % accumulation_steps == 0:  #Not used? # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>threshold) == (y_batch[:,0]>threshold).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)

torch.save(model.state_dict(), path)

100%|██████████| 407873900/407873900 [00:34<00:00, 11738313.84B/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


In [ ]:
#Re-load model from file
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=len(y_columns))
model.load_state_dict(torch.load(path ))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [ ]:
#Validation
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [ ]:
# # From baseline kernel
# #Not used 27/05/19
# from sklearn import metrics
# from sklearn import model_selection
# def calculate_overall_auc(df, model_name):
#     true_labels = df[TOXICITY_COLUMN]>threshold
#     predicted_labels = df[model_name]
#     return metrics.roc_auc_score(true_labels, predicted_labels)

# def power_mean(series, p):
#     total = sum(np.power(series, p))
#     return np.power(total / len(series), 1 / p)

# def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
#     bias_score = np.average([
#         power_mean(bias_df[SUBGROUP_AUC], POWER),
#         power_mean(bias_df[BPSN_AUC], POWER),
#         power_mean(bias_df[BNSP_AUC], POWER)
#     ])
#     return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



# SUBGROUP_AUC = 'subgroup_auc'
# BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
# BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

# def compute_auc(y_true, y_pred):
#     try:
#         return metrics.roc_auc_score(y_true, y_pred)
#     except ValueError:
#         return np.nan

# def compute_subgroup_auc(df, subgroup, label, model_name):
#     subgroup_examples = df[df[subgroup]>threshold]
#     return compute_auc((subgroup_examples[label]>threshold), subgroup_examples[model_name])

# def compute_bpsn_auc(df, subgroup, label, model_name):
#     """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
#     subgroup_negative_examples = df[(df[subgroup]>threshold) & (df[label]<=threshold)]
#     non_subgroup_positive_examples = df[(df[subgroup]<=threshold) & (df[label]>threshold)]
#     examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
#     return compute_auc(examples[label]>threshold, examples[model_name])

# def compute_bnsp_auc(df, subgroup, label, model_name):
#     """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
#     subgroup_positive_examples = df[(df[subgroup]>threshold) & (df[label]>threshold)]
#     non_subgroup_negative_examples = df[(df[subgroup]<=threshold) & (df[label]<=threshold)]
#     examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
#     return compute_auc(examples[label]>threshold, examples[model_name])

# def compute_bias_metrics_for_model(dataset,
#                                    subgroups,
#                                    model,
#                                    label_col,
#                                    include_asegs=False):
#     """Computes per-subgroup metrics for all subgroups and one model."""
#     records = []
#     for subgroup in subgroups:
#         record = {
#             'subgroup': subgroup,
#             'subgroup_size': len(dataset[dataset[subgroup]>threshold])
#         }
#         record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
#         record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
#         record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
#         records.append(record)
#     return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)


In [ ]:
# MODEL_NAME = 'model1'

In [ ]:
# test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
# print(test_df[MODEL_NAME].describe())

ValueError: ignored

In [ ]:
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
print(bias_metrics_df)
final = get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))
print(final)

In [ ]:
sentence = '99.9 percent liberal voices on the air. All right. Tucker, great job. All right. Welcome to HANNITY. All right. Buckle up tonight. I have a special message for Jim Comey. Now, for nearly two years, the United States of America, we have somehow managed to survive without the dedicated stewardship of our righteous moral savior, the ever so virtuous and honorable James Comey. Now, thankfully, he\'s back and telling all of us, you know, we are the stupid, smelly, Walmart shoppers of America. And, of course, his colleague and friend Peter Strzok, that\'s how he refers to us. He\'s now telling us how to behave as the Mueller witch hunt nears its conclusion. I will get to that.'

input = convert_lines([sentence], MAX_SEQUENCE_LENGTH, tokenizer)

print(input.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



1
(1, 128)


In [ ]:
input = torch.tensor(input, dtype=torch.long)
pred = model(input.to(device), attention_mask=(input>0).to(device), labels=None)
new = torch.sigmoid(pred)
print(new)

tensor([[0.5376]], device='cuda:0')


## Test Set

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path = '/content/gdrive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/test_data/'
filename = 'test_public_expanded.csv'

threshold = 0

In [ ]:
import pandas as pd
test_data  = pd.read_csv(path+filename, usecols=['comment_text', 'toxicity']) 

In [ ]:
!pip install pytorch-pretrained-bert

In [ ]:
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
from tqdm import tqdm, tqdm_notebook
import numpy as np
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [ ]:
# Make sure all comment_text values are strings
MAX_SEQUENCE_LENGTH=128
test_data['comment_text'] = test_data['comment_text'].astype(str) 

sequences = convert_lines(test_data["comment_text"],MAX_SEQUENCE_LENGTH,tokenizer)
test_data=test_data.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['toxicity']

test_data = test_data.drop(['comment_text'],axis=1)
# convert toxicity to 0,1
test_data['toxicity']=(test_data['toxicity']>threshold).astype(float)

X_test = sequences                
y_test = test_data[y_columns].values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



15656


In [ ]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F

In [ ]:
#Re-load model from file

path = "/content/gdrive/My Drive/Incivility/Perspective/Perspective_BERT/bert_pytorch_0.5_larger.bin"
# path = "/content/gdrive/My Drive/Incivility/Perspective/Perspective_BERT/Final/Data/bert_pytorch_0_thresh.bin"
# path = "/content/gdrive/My Drive/Incivility/Perspective/Perspective_BERT/Retrained/bert_pytorch_0.5_retrained.bin"

model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=len(y_columns))
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
device=torch.device('cuda')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [ ]:
for param in model.parameters():
    param.requires_grad=False
model.eval()
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test,dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(test_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    test_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
import csv
path = '/content/gdrive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/test_data/'
filename = 'test_public_expanded.csv'
test_data  = pd.read_csv(path+filename, usecols=['comment_text', 'toxicity']) 
with open('/content/gdrive/My Drive/Incivility/Perspective/Perspective_BERT/BERT_test_set_0.5_larger.tsv', 'w') as w:
    writer = csv.writer(w, delimiter='\t')
    writer.writerow(['comment', 'label', 'BERT'])
    for text, label, pred in zip(test_data["comment_text"], test_data["toxicity"], test_preds):
        writer.writerow([text, label, torch.sigmoid(torch.tensor(pred)).item()])